# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.utilities import get_primary_metrics
from azureml.widgets import RunDetails
import joblib
import json
import os
import pandas as pd
import requests

In [2]:
# Set pandas options to be able to display all columns
pd.set_option("display.max_columns", 35)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

I am using the IBM HR Analytics Employee Attrition & Performance dataset from Kaggle (https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset) for the binary classification task of predicting whether an employee is at risk of leaving the company, a phenomenon that is called attrition. The dataset contains the target class "Attrition" as well as demographic, job-specific, geographic and socioeconomic feature columns. It is a fictional dataset created by IBM Data Scientists. In total the dataset has 1470 rows and 35 columns. I have set aside 20% for testing and have registered 80% as training dataset in the AML workspace (check **explorative_data_analysis.ipynb** for details).

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
# Create a workspace object
ws = Workspace.from_config()

# Create an experiment
experiment_name = "attrition-automl"
experiment = Experiment(ws, experiment_name)

# Retrieve the training data from the workspace
dataset = Dataset.get_by_name(ws, name="attrition_train")
df = dataset.to_pandas_dataframe()

# Show the first rows
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,True,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,True,True,11,3,1,80,0,8,0,1,6,4,0,5
1,49,False,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,True,False,23,4,4,80,1,10,3,3,10,7,1,7
2,37,True,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,True,True,15,3,2,80,0,7,3,3,0,0,0,0
3,33,False,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,True,True,11,3,3,80,0,8,3,3,8,7,3,0
4,27,False,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,True,False,12,3,4,80,1,6,3,3,2,2,2,2


In [4]:
# Split data further into train and validation set
dataset_train, dataset_val = dataset.random_split(0.8, seed=42)

## Compute Target

In [5]:
compute_cluster_name = "cpu-cluster"

# Retrieve the compute target if already available or create a new one
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, using it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, using it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.
- "enable_early_stopping" is set to true to discard non-promising training runs.
- "experiment_timeout_minutes" is set to 90 to give the automl experiment 90min of training time to train a powerful model but stay within resource constraints. 
- "max_concurrent_iterations" is set to 4 as this number should be smaller than the node count of the cluster (which is 5) but should be leveraged to a maximum for parallel model training.
- AUC_weighted is used as "primary_metric" because it can account for imbalanced classes (which we face here as analyzed in **explorative_data_analysis.ipynb**) in contrast to simple accuracy.
- Similar to the baseline_model in **baseline_model.ipynb**, 20% of the data will be used for validation ("validation_data") and 80% for training ("training_data").
- "featurization" is set to auto for automatic featurization.

In [6]:
# Check available primary metrics for classification
get_primary_metrics("classification")

['norm_macro_recall',
 'accuracy',
 'AUC_weighted',
 'average_precision_score_weighted',
 'precision_score_weighted']

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "enable_early_stopping": True,
    "experiment_timeout_minutes": 90,
    "max_concurrent_iterations": 4,
    "primary_metric" : "AUC_weighted"
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target = compute_target,
    task = "classification",
    training_data = dataset_train,
    validation_data = dataset_val,
    label_column_name = "Attrition",
    path = "./automl",
    featurization = "auto",
    **automl_settings
)

In [8]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
automl_run.wait_for_completion(show_output=True)


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number 

{'runId': 'AutoML_fb44cb4e-956d-4d7b-bd6f-6230db6ae20c',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-17T20:44:14.540983Z',
 'endTimeUtc': '2020-12-17T21:04:53.338688Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"attrition-automl","subscription_id":"bf088f59-f015-4332-bd36-54b988be7c90","resource_group":"sbirk-aml-rg","workspace_name":"sbirk-aml-ws","region":"westeurope","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_automl_run, automl_model = automl_run.get_output()
print(best_automl_run)
print(automl_model)

Run(Experiment: attrition-automl,
Id: AutoML_fb44cb4e-956d-4d7b-bd6f-6230db6ae20c_45,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.01,
                                                                                                    min_weight_fraction_leaf=0.0,
        

In [12]:
#TODO: Save the best model
os.makedirs("./models", exist_ok=True)
joblib.dump(automl_model, "models/automl_model.pkl")

['models/automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [13]:
# Download the inference file
model_name = best_automl_run.properties["model_name"]
script_file_name = "inference/score.py"
best_automl_run.download_file("outputs/scoring_file_v_1_0_0.py", "inference/score.py")

In [14]:
# Register the best model
description = "AutoML model trained on attrition data to predict if an employee will leave the company"
tags = None
model = automl_run.register_model(model_name = model_name, description = description, tags = tags)

print(automl_run.model_id)

AutoMLfb44cb4e945


In [22]:
# Deploy the model as a web service on Azure Container Instance
inference_config = InferenceConfig(entry_script=script_file_name)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1,
    memory_gb = 1,
    tags = {"data": "attrition", 'type': "automl_model"},
    description = "Attrition prediction web service based on automl model"
)

aci_service_name = "attrition-prediction-web-service"
print(aci_service_name)

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)

aci_service.wait_for_deployment(True)
print(aci_service.state)

attrition-prediction-web-service
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [49]:
# Load unseen test data and split target and feature columns
df = pd.read_csv("./data/attrition_test.csv")
X_test = df.loc[:, df.columns != "Attrition"].reset_index(drop=True)
y_test = df.loc[:, "Attrition"].reset_index(drop=True)

In [50]:
# Send a request to the web service with some sample rows from the test set
X_test_json = X_test[:50].to_json(orient="records")
data = "{\"data\": " + X_test_json + "}"
headers = {'Content-Type': 'application/json'}

resp = requests.post(aci_service.scoring_uri, data, headers=headers)
y_test_pred = json.loads(json.loads(resp.text))['result']

In [51]:
# Compare prediction results and actual targets
pd.concat([y_test[:50], pd.DataFrame({"AttritionPrediction": y_test_pred}).replace({"AttritionPrediction": {True: "Yes", False: "No"}}), X_test[:50]], axis=1)

,Attrition,AttritionPrediction,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,...,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,No,No,49,Travel_Rarely,301,Research & Development,22,4,Other,1,1655,1,Female,72,3,4,Research Director,...,16413,3498,3,Y,No,16,3,2,80,2,27,2,3,4,2,1,2
1,No,No,50,Travel_Rarely,813,Research & Development,17,5,Life Sciences,1,1656,4,Female,50,2,3,Research Director,...,13269,21981,5,Y,No,15,3,3,80,3,19,3,3,14,11,1,11
2,No,Yes,20,Travel_Rarely,1141,Sales,2,3,Medical,1,1657,3,Female,31,3,1,Sales Representative,...,2783,13251,1,Y,No,19,3,1,80,0,2,3,3,2,2,2,2
3,No,No,34,Travel_Rarely,1130,Research & Development,3,3,Life Sciences,1,1658,4,Female,66,3,2,Research Scientist,...,5433,19332,1,Y,No,12,3,3,80,1,11,2,3,11,8,7,9
4,No,Yes,36,Travel_Rarely,311,Research & Development,7,3,Life Sciences,1,1659,1,Male,77,3,1,Laboratory Technician,...,2013,10950,2,Y,No,11,3,3,80,0,15,4,3,4,3,1,3
5,No,No,49,Travel_Rarely,465,Research & Development,6,1,Life Sciences,1,1661,3,Female,41,2,4,Healthcare Representative,...,13966,11652,2,Y,Yes,19,3,2,80,1,30,3,3,15,11,2,12
6,No,No,36,Non-Travel,894,Research & Development,1,4,Medical,1,1662,4,Female,33,2,2,Manufacturing Director,...,4374,15411,0,Y,No,15,3,3,80,0,4,6,3,3,2,1,2
7,No,No,36,Travel_Rarely,1040,Research & Development,3,2,Life Sciences,1,1664,4,Male,79,4,2,Healthcare Representative,...,6842,26308,6,Y,No,20,4,1,80,1,13,3,3,5,4,0,4
8,No,No,54,Travel_Rarely,584,Research & Development,22,5,Medical,1,1665,2,Female,91,3,4,Manager,...,17426,18685,3,Y,No,25,4,3,80,1,36,6,3,10,8,4,7
9,No,No,43,Travel_Rarely,1291,Research & Development,15,2,Life Sciences,1,1666,3,Male,65,2,4,Research Director,...,17603,3525,1,Y,No,24,4,1,80,1,14,3,3,14,10,6,11


TODO: In the cell below, print the logs of the web service and delete the service

In [26]:
# Get logs from the web service
aci_service.get_logs()

'2020-12-17T21:21:25,115276700+00:00 - nginx/run \n2020-12-17T21:21:25,141352400+00:00 - iot-server/run \n2020-12-17T21:21:25,138627200+00:00 - gunicorn/run \n/usr/sbin/nginx: /azureml-envs/azureml_e1251fbb90896de9de2cde1255eba7bb/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_e1251fbb90896de9de2cde1255eba7bb/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_e1251fbb90896de9de2cde1255eba7bb/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_e1251fbb90896de9de2cde1255eba7bb/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_e1251fbb90896de9de2cde1255eba7bb/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-12-17T21:21:25,145892500+00:00 - rsyslog/run 

In [20]:
# Delete the web service
aci_service.delete()